# From 1 to Infinity

## Data

数据接口：Ashare

简易的免安装版本，可获取实时行情

需要的库：json,requests,datetime,numpy,pandas

In [2]:
from  Ashare import *          #股票数据库    https://github.com/mpquant/Ashare
from  MyTT import *            #myTT麦语言工具函数指标库  https://github.com/mpquant/MyTT
    
# 证券代码兼容多种格式 通达信，同花顺，聚宽
# sh000001 (000001.XSHG)    sz399006 (399006.XSHE)   sh600519 (600519.XSHG) 

df=get_price('sh000001',frequency='1d',count=120)      #默认获取今天往前120天的日线行情

'''
count的设置
----------
每年约250个交易日，最早的交易记录可以追朔到1991年上交所成立，至2023年底约33年，8250个交易日
设定为10000足以应对绝大多数情况，并在未来5年间代码仍然适用

frequency的设置
--------------
'1d'日线
'1w'周线
'1M'月线
'1m'1分钟线
'5m'5分钟线
'15m'15分钟线
'30m'30分钟线
'60m'60分钟线
'''
print('上证指数日线行情\n',df.tail(5))

#-------有数据了，下面开始正题 -------------
CLOSE=df.close.values;         OPEN=df.open.values           #基础数据定义，只要传入的是序列都可以  Close=df.close.values 
HIGH=df.high.values;           LOW=df.low.values             #例如  CLOSE=list(df.close) 都是一样

MA5=MA(CLOSE,5)                                #获取5日均线序列
MA10=MA(CLOSE,10)                              #获取10日均线序列
up,mid,lower=BOLL(CLOSE)                       #获取布林带指标数据

上证指数日线行情
                 open      high       low     close        volume
                                                                
2023-07-06  3215.863  3229.480  3199.716  3205.575  2.461190e+10
2023-07-07  3197.472  3212.047  3189.211  3196.609  2.556885e+10
2023-07-10  3208.735  3216.535  3192.658  3203.696  2.365604e+10
2023-07-11  3211.122  3221.680  3200.385  3221.367  2.276005e+10
2023-07-12  3220.376  3224.331  3193.437  3196.134  2.714234e+10


数据接口：AKshare


In [2]:
import akshare as ak
import pandas as pd
import numpy as np
from tqdm import tqdm
import lightgbm as lgb


In [6]:
# 获取指数成分股

# hs300 = ak.index_stock_cons_weight_csindex("000300")  #获取沪深300成分股权重数据
# hs300.to_csv('hs300.csv',encoding='utf-8-sig')        #保存到本地
# zz500 = ak.index_stock_cons_weight_csindex("000905")  #获取中证500成分股权重数据
# zz500.to_csv('zz500.csv',encoding='utf-8-sig')        #保存到本地
# zz800 = ak.index_stock_cons_weight_csindex("000906")  #获取中证800成分股权重数据
# zz800.to_csv('zz800.csv',encoding='utf-8-sig')        #保存到本地
# zz1000 = ak.index_stock_cons_weight_csindex("000852") #获取中证1000成分股权重数据
# zz1000.to_csv('zz1000.csv',encoding='utf-8-sig')      #保存到本地

hs300 = pd.read_csv('../data/index/hs300.csv',encoding='utf-8-sig') #读取本地数据
zz500 = pd.read_csv('../data/index/zz500.csv',encoding='utf-8-sig') #读取本地数据
zz800 = pd.read_csv('../data/index/zz800.csv',encoding='utf-8-sig') #读取本地数据
zz1000 = pd.read_csv('../data/index/zz1000.csv',encoding='utf-8-sig') #读取本地数据

In [16]:
# 获取单支指数成分股历史数据
single_df = ak.stock_zh_a_hist(symbol="300014", period="daily", start_date="19900101", end_date='20301231', adjust="hfq")
single_df

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2009-10-30,38.00,43.28,57.00,36.00,157364,6.627058e+08,116.67,140.44,25.28,89.41
1,2009-11-02,38.95,38.95,41.22,38.95,59404,2.328985e+08,5.24,-10.00,-4.33,33.75
2,2009-11-03,38.00,37.77,39.93,36.00,54480,2.078191e+08,10.09,-3.03,-1.18,30.95
3,2009-11-04,38.20,39.86,40.50,38.18,50126,1.972197e+08,6.14,5.53,2.09,28.48
4,2009-11-05,39.10,39.92,41.77,38.32,47094,1.881825e+08,8.66,0.15,0.06,26.76
...,...,...,...,...,...,...,...,...,...,...,...
3263,2023-07-06,1026.90,1003.69,1028.10,998.39,160065,9.349515e+08,2.88,-2.57,-26.46,0.87
3264,2023-07-07,998.39,981.49,1001.81,981.32,126405,7.246175e+08,2.04,-2.21,-22.20,0.68
3265,2023-07-10,984.74,1023.32,1045.68,984.74,251476,1.494632e+09,6.21,4.26,41.83,1.36
3266,2023-07-11,1019.39,1028.27,1031.17,1004.71,138578,8.173158e+08,2.59,0.48,4.95,0.75


In [8]:
def log_diff(df):
    df.replace(0, 0.01, inplace=True)
    diffdf = np.log(df).loc[1:,:] - np.log(df.shift(1)).loc[1:,:]
    return diffdf

def cut(df, train_period=5, pred_period=1):
    '''
    train_period: 用于训练的数据长度，默认前5个交易日
    pred_period: 预测几日后的结果，默认第二天
    '''
    n_features = df.shape[1]
    roll = pd.concat([df.shift(i) for i in range(pred_period,pred_period+train_period)], axis=1)
    roll = pd.concat([df, roll], axis=1).dropna()
    train = roll.iloc[:, n_features:]
    pred = roll['收盘'].iloc[:,0]
    return train, pred

In [13]:
# 获取全部指数成分股历史数据
X_train = pd.DataFrame()
y_train = pd.DataFrame()
for code in tqdm(zz1000['成分券代码']):
    if len(str(code)) < 6:
        code = '0'*(6-len(str(code))) + str(code)
    single_df = ak.stock_zh_a_hist(symbol=str(code), period="daily", start_date="19900101", end_date='20301231', adjust="hfq")
    X, y = cut(log_diff(single_df.drop(columns=['日期','涨跌幅','涨跌额','振幅'])), train_period=5, pred_period=1)
    X_train = pd.concat([X_train, X])
    y_train = pd.concat([y_train, y])

  0%|          | 0/1000 [00:00<?, ?it/s]

  7%|▋         | 69/1000 [01:34<22:01,  1.42s/it]/Users/ivan/miniforge3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
 10%|█         | 100/1000 [02:26<21:37,  1.44s/it]/Users/ivan/miniforge3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
 13%|█▎        | 129/1000 [03:11<23:25,  1.61s/it]/Users/ivan/miniforge3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
 13%|█▎        | 130/1000 [03:12<21:58,  1.52s/it]/Users/ivan/miniforge3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
 16%|█▌        | 162/1000 [04:06<22:09,  1.59s/it]/Users/ivan/miniforge3/lib/python3.9/site-packa

In [6]:
X_train.to_csv('../data/X_train_zz1000.csv',encoding='utf-8-sig',index=False)        #保存到本地
y_train.to_csv('../data/y_train_zz1000.csv',encoding='utf-8-sig',index=False)        #保存到本地

# X_train = pd.read_csv('X_train_hs300.csv',encoding='utf-8-sig') #读取本地数据
# y_train = pd.read_csv('y_train_hs300.csv',encoding='utf-8-sig') #读取本地数据

OSError: [Errno 22] Invalid argument

In [69]:
hs300

,Unnamed: 0,日期,指数代码,指数名称,指数英文名称,成分券代码,成分券名称,成分券英文名称,交易所,交易所英文名称,权重
0,0,2023-06-30,300,沪深300,CSI 300,300014,亿纬锂能,Eve Energy Co Ltd,深圳证券交易所,Shenzhen Stock Exchange,0.393
1,1,2023-06-30,300,沪深300,CSI 300,300015,爱尔眼科,Aier Eye Hospital Group Co Ltd,深圳证券交易所,Shenzhen Stock Exchange,0.458
2,2,2023-06-30,300,沪深300,CSI 300,600150,中国船舶,China CSSC Holdings Limited,上海证券交易所,Shanghai Stock Exchange,0.389
3,3,2023-06-30,300,沪深300,CSI 300,600188,兖矿能源,Yankuang Energy Group Company Limited,上海证券交易所,Shanghai Stock Exchange,0.145
4,4,2023-06-30,300,沪深300,CSI 300,600039,四川路桥,Sichuan Road&Bridge Co Ltd,上海证券交易所,Shanghai Stock Exchange,0.136
...,...,...,...,...,...,...,...,...,...,...,...
295,295,2023-06-30,300,沪深300,CSI 300,688187,时代电气,"Zhuzhou CRRC Times Electric Co., Ltd.",上海证券交易所,Shanghai Stock Exchange,0.058
296,296,2023-06-30,300,沪深300,CSI 300,601868,中国能建,China Energy Engineering Corporation Limited,上海证券交易所,Shanghai Stock Exchange,0.161
297,297,2023-06-30,300,沪深300,CSI 300,600941,中国移动,China Mobile Limited,上海证券交易所,Shanghai Stock Exchange,0.446
298,298,2023-06-30,300,沪深300,CSI 300,1289,龙源电力,China Longyuan Power Group Corporation Limited,深圳证券交易所,Shenzhen Stock Exchange,0.018
